In [2]:
import pandas as pd
import numpy as np
from collections import Counter

# Step 1: Define data
data = {
    "Unit Operation": [
        "Corn Cleaning", "Corn Cooking", "Corn Soaking", "Washing & Milling",
        "Sheeting", "Toasting", "Proofing", "Frying", "Seasoning"
    ],
    "WIP (lbs)": [650, 3000, 36000, 1500, 300, 300, 750, 300, 600],
    "Cost per lb ($)": [0.10, 0.10, 0.11, 0.11, 0.11, 0.12, 0.12, 0.60, 0.91],
    "Dwell Time (hr)": [0.1, 1.5, 13.5, 0.5, 0.1, 0.1, 0.25, 0.1, 0.2],
    "MTTR (hr)": [0.01, 2.0, 13.5, 1.2, 0.01, 2.0, 0.25, 3.8, 0.25],
    "Mitigation Cost ($)": [10611, 18778, 6800, 18486, 18486, 10611, 10611, 37574, 10611]
}

df = pd.DataFrame(data)

# Constants
NUM_YEARS = 5
BUDGET = 25000
EVENTS_PER_YEAR = 36
NUM_SIMULATIONS = 1000

# Step 2: Derived metrics
df["Waste/Event ($)"] = df["WIP (lbs)"] * df["Cost per lb ($)"]
df["Expected Annual Loss ($)"] = EVENTS_PER_YEAR * df["Waste/Event ($)"]
df["ROI"] = df["Expected Annual Loss ($)"] / df["Mitigation Cost ($)"]
df["CIS"] = df["Dwell Time (hr)"] * df["WIP (lbs)"] * df["MTTR (hr)"]

# Sort
roi_sorted = df.sort_values(by="ROI", ascending=False).reset_index(drop=True)
toc_sorted = df.sort_values(by="CIS", ascending=False).reset_index(drop=True)

# Step 3: 5-year planning function
def plan_strategy(df_sorted):
    remaining = df_sorted.copy()
    plan = []
    for year in range(1, NUM_YEARS + 1):
        cost = 0
        units = []
        for i, row in remaining.iterrows():
            if cost + row["Mitigation Cost ($)"] <= BUDGET:
                cost += row["Mitigation Cost ($)"]
                units.append(row["Unit Operation"])
        plan.append({"Year": year, "Units": units, "Total Cost": cost})
        remaining = remaining[~remaining["Unit Operation"].isin(units)]
    return pd.DataFrame(plan)

roi_plan_df = plan_strategy(roi_sorted)
toc_plan_df = plan_strategy(toc_sorted)

# Step 4: Sensitivity simulation
roi_freq = {i: [] for i in range(1, 6)}
roi_rois = {i: [] for i in range(1, 6)}
toc_freq = {i: [] for i in range(1, 6)}
toc_cis = {i: [] for i in range(1, 6)}

for _ in range(NUM_SIMULATIONS):
    sim = df.copy()
    sim["Sim Waste"] = sim["Waste/Event ($)"] * np.random.uniform(0.8, 1.2, len(sim))
    sim["Sim Annual Loss"] = EVENTS_PER_YEAR * sim["Sim Waste"]
    sim["Sim ROI"] = sim["Sim Annual Loss"] / sim["Mitigation Cost ($)"]
    roi_sorted_sim = sim.sort_values(by="Sim ROI", ascending=False).reset_index(drop=True)

    used_units = set()
    for year in range(1, 6):
        cost = 0
        selected = []
        roi_sum = 0
        for _, row in roi_sorted_sim.iterrows():
            if row["Unit Operation"] not in used_units and cost + row["Mitigation Cost ($)"] <= BUDGET:
                selected.append(row["Unit Operation"])
                roi_sum += row["Sim ROI"]
                cost += row["Mitigation Cost ($)"]
                used_units.add(row["Unit Operation"])
        roi_freq[year].extend(selected)
        if selected:
            roi_rois[year].append(roi_sum / len(selected))

    # ToC (fixed order)
    used = set()
    for year in range(1, 6):
        cost = 0
        selected = []
        cis_sum = 0
        for _, row in toc_sorted.iterrows():
            if row["Unit Operation"] not in used and cost + row["Mitigation Cost ($)"] <= BUDGET:
                selected.append(row["Unit Operation"])
                cis_sum += row["CIS"]
                cost += row["Mitigation Cost ($)"]
                used.add(row["Unit Operation"])
        toc_freq[year].extend(selected)
        if selected:
            toc_cis[year].append(cis_sum / len(selected))

# Step 5: Reporting results
def summarize_selection(selection_tracker):
    summary = []
    for year, selections in selection_tracker.items():
        counts = Counter(selections)
        for unit, count in counts.items():
            summary.append({
                "Year": year,
                "Unit Operation": unit,
                "Selection Frequency (%)": round(count / NUM_SIMULATIONS * 100, 2)
            })
    return pd.DataFrame(summary)

def summarize_metrics(metric_tracker, label):
    return pd.DataFrame([{
        "Year": year,
        f"Mean {label}": np.mean(values),
        f"Min {label}": np.min(values),
        f"Max {label}": np.max(values),
        f"Std Dev {label}": np.std(values)
    } for year, values in metric_tracker.items()])

roi_freq_df = summarize_selection(roi_freq)
toc_freq_df = summarize_selection(toc_freq)
roi_perf_df = summarize_metrics(roi_rois, "ROI")
toc_perf_df = summarize_metrics(toc_cis, "CIS")

# View results
from IPython.display import display

print("=== ROI-Based 5-Year Plan ===")
display(roi_plan_df)

print("=== ToC-Based 5-Year Plan ===")
display(toc_plan_df)

print("=== ROI Sensitivity Selection Frequencies ===")
display(roi_freq_df)

print("=== ToC Sensitivity Selection Frequencies ===")
display(toc_freq_df)

print("=== ROI Performance Summary ===")
display(roi_perf_df)

print("=== ToC Performance Summary ===")
display(toc_perf_df)

=== ROI-Based 5-Year Plan ===


,Year,Units,Total Cost
0,1,"[Corn Soaking, Seasoning]",17411
1,2,[Corn Cooking],18778
2,3,[Washing & Milling],18486
3,4,"[Proofing, Corn Cleaning]",21222
4,5,[Toasting],10611


=== ToC-Based 5-Year Plan ===


,Year,Units,Total Cost
0,1,"[Corn Soaking, Toasting]",17411
1,2,[Corn Cooking],18778
2,3,[Washing & Milling],18486
3,4,"[Proofing, Seasoning]",21222
4,5,[Corn Cleaning],10611


=== ROI Sensitivity Selection Frequencies ===


...

,Year,Unit Operation,Selection Frequency (%)
0,1,Corn Soaking,100.0
1,1,Toasting,100.0
2,2,Corn Cooking,100.0
3,3,Washing & Milling,100.0
4,4,Proofing,100.0
5,4,Seasoning,100.0
6,5,Corn Cleaning,100.0


=== ROI Performance Summary ===


,Year,Mean ROI,Min ROI,Max ROI,Std Dev ROI
0,1,11.380366,9.207601,13.622824,1.204904
1,2,0.578556,0.460944,0.689958,0.066907
2,3,0.314598,0.223134,0.385540,0.041828
3,4,0.270306,0.212991,0.362816,0.028892
4,5,0.121652,0.097747,0.146521,0.014152


=== ToC Performance Summary ===


,Year,Mean CIS,Min CIS,Max CIS,Std Dev CIS
0,1,3.280530e+06,3.280530e+06,3.280530e+06,0.000000e+00
1,2,9.000000e+03,9.000000e+03,9.000000e+03,0.000000e+00
2,3,9.000000e+02,9.000000e+02,9.000000e+02,0.000000e+00
3,4,3.843750e+01,3.843750e+01,3.843750e+01,0.000000e+00
4,5,6.500000e-01,6.500000e-01,6.500000e-01,2.220446e-16
